# Conv2d + Conv1d Modified tests

In [1]:
import sys 
sys.path.append("../")

import torch 
import torch.nn as nn
from torchsummary import summary

# from Conv1d_NN import Conv1d_NN
from Conv2d_NN import Conv2d_NN

from models import * 
from train import * 
from dataset import * 
from pixelshuffle import * 



torch.Size([32, 32, 40])


## Testing Conv2d

In [2]:
# Data 
# Import necessary libraries
from torchvision import datasets
data_folder = '~/data/MNIST'
mnist_train = datasets.MNIST(data_folder, download=True, train=True)
x_train, y_train = mnist_train.data, mnist_train.targets
print(x_train.shape, y_train.shape)

from torch.utils.data import Dataset
class MNISTDataset(Dataset):
   def __init__(self, x, y):
      x = x.float()/255 # Data rescaling
      self.x, self.y = x, y
   def __len__(self):
      return len(self.x)
   def __getitem__(self, ix):
      x, y = self.x[ix], self.y[ix]
      return x.to('cpu'), y.to('cpu')

train_dataset = MNISTDataset(x_train, y_train)

from torch.utils.data import DataLoader
train_dl = DataLoader(train_dataset, batch_size=32, shuffle=True)

torch.Size([60000, 28, 28]) torch.Size([60000])


In [3]:
# Model 
conv2d_nn = nn.Sequential(
   Conv2d_NN(
      in_channels=1,
      out_channels=5,
      K=3,
      stride=3,
      padding=0,
      shuffle_scale=2
   ), 
   nn.ReLU(),
   Conv2d_NN(
      in_channels=5,
      out_channels=10,
      K=3,
      stride=3,
      padding=0,
      shuffle_scale=2
   ),
   nn.ReLU(),
   Conv2d_NN(
      in_channels=10,
      out_channels=20,
      K=3,
      stride=3,
      padding=0,
      shuffle_scale=2
   ),
   nn.Flatten(), 
   nn.Linear(15680, 10)
   
).to('cpu')
   

from torchsummary import summary
summary(conv2d_nn, (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1               [-1, 4, 196]               0
            Conv1d-2              [-1, 20, 196]             260
              ReLU-3              [-1, 20, 196]               0
         Conv1d_NN-4              [-1, 20, 196]               0
         Conv2d_NN-5            [-1, 5, 28, 28]               0
              ReLU-6            [-1, 5, 28, 28]               0
           Flatten-7              [-1, 20, 196]               0
            Conv1d-8              [-1, 40, 196]           2,440
              ReLU-9              [-1, 40, 196]               0
        Conv1d_NN-10              [-1, 40, 196]               0
        Conv2d_NN-11           [-1, 10, 28, 28]               0
             ReLU-12           [-1, 10, 28, 28]               0
          Flatten-13              [-1, 40, 196]               0
           Conv1d-14              [-1, 

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Assuming Conv2d_NN is defined elsewhere in your code
# Define your model here as per the provided structure

# Load and preprocess data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

# Training function
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')

# Accuracy evaluation function
def evaluate_accuracy(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Accuracy on test set: {accuracy}%')
    return accuracy

# Initialize model, loss, and optimizer
model = conv2d_nn
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

# Train the model
train_model(model, train_loader, criterion, optimizer, num_epochs=10)

# Evaluate accuracy
evaluate_accuracy(model, test_loader)


Epoch 1, Loss: 0.31437504275648326
Epoch 2, Loss: 0.16612954330898677
Epoch 3, Loss: 0.12960919333613916
Epoch 4, Loss: 0.10548594566846468
Epoch 5, Loss: 0.0941099752860664
Epoch 6, Loss: 0.07940264251457949
Epoch 7, Loss: 0.06897804607798073
Epoch 8, Loss: 0.06303184081652342
Epoch 9, Loss: 0.05436393609473994
Epoch 10, Loss: 0.04747006030659428
Accuracy on test set: 95.87%


95.87

In [9]:
## Do a sample regular CNN 

cnn = nn.Sequential(
   nn.Conv2d(
      in_channels=1,
      out_channels=5,
      kernel_size=3
   ), 
   nn.ReLU(),
   nn.Conv2d(
      in_channels=5,
      out_channels=10,
      kernel_size=3
   ), 
   nn.ReLU(),
   nn.Conv2d(
      in_channels=10,
      out_channels=20,
      kernel_size=3
   ), 
   nn.ReLU(),
   nn.Flatten(), 
   nn.Linear(9680, 10)
   
).to('cpu')
   

from torchsummary import summary
summary(cnn, (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 5, 26, 26]              50
              ReLU-2            [-1, 5, 26, 26]               0
            Conv2d-3           [-1, 10, 24, 24]             460
              ReLU-4           [-1, 10, 24, 24]               0
            Conv2d-5           [-1, 20, 22, 22]           1,820
              ReLU-6           [-1, 20, 22, 22]               0
           Flatten-7                 [-1, 9680]               0
            Linear-8                   [-1, 10]          96,810
Total params: 99,140
Trainable params: 99,140
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.36
Params size (MB): 0.38
Estimated Total Size (MB): 0.74
----------------------------------------------------------------


In [10]:
# Train the model
train_model(cnn, train_loader, criterion, optimizer, num_epochs=10)

# Evaluate accuracy
evaluate_accuracy(cnn, test_loader)

Epoch 1, Loss: 2.305797335181409
Epoch 2, Loss: 2.305781441965083
Epoch 3, Loss: 2.3057882788338895
Epoch 4, Loss: 2.305787814451433
Epoch 5, Loss: 2.3057871871411417
Epoch 6, Loss: 2.3057813685077595
Epoch 7, Loss: 2.305788746774832
Epoch 8, Loss: 2.30578144018584
Epoch 9, Loss: 2.3057826767598133
Epoch 10, Loss: 2.305790897879773
Accuracy on test set: 8.51%


8.51

In [11]:
cnn2 = nn.Sequential(
   nn.Conv2d(1, 64, 3), 
   nn.MaxPool2d(2),
   nn.ReLU(),
   nn.Conv2d(64, 128, 3),
   nn.MaxPool2d(2),
   nn.ReLU(),
   nn.Flatten(), 
   nn.Linear(3200, 200), 
   nn.ReLU(), 
   nn.Linear(200, 10)
).to('cpu')

summary(cnn2, (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 26, 26]             640
         MaxPool2d-2           [-1, 64, 13, 13]               0
              ReLU-3           [-1, 64, 13, 13]               0
            Conv2d-4          [-1, 128, 11, 11]          73,856
         MaxPool2d-5            [-1, 128, 5, 5]               0
              ReLU-6            [-1, 128, 5, 5]               0
           Flatten-7                 [-1, 3200]               0
            Linear-8                  [-1, 200]         640,200
              ReLU-9                  [-1, 200]               0
           Linear-10                   [-1, 10]           2,010
Total params: 716,706
Trainable params: 716,706
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.69
Params size (MB): 2.73
Estimated T

In [12]:
# Train the model
train_model(cnn2, train_loader, criterion, optimizer, num_epochs=10)

# Evaluate accuracy
evaluate_accuracy(cnn2, test_loader)

Epoch 1, Loss: 2.3038571015604017
Epoch 2, Loss: 2.3038535082518163
Epoch 3, Loss: 2.303859902851617
Epoch 4, Loss: 2.3038589189301675
Epoch 5, Loss: 2.3038686541860294
Epoch 6, Loss: 2.303862106825497
Epoch 7, Loss: 2.3038635040397075
Epoch 8, Loss: 2.3038639770642018
Epoch 9, Loss: 2.30386119432795
Epoch 10, Loss: 2.3038573717511794
Accuracy on test set: 6.79%


6.79